In [1]:
import tensorflow as tf
import numpy as np
import librosa
import random

# Simulate MFCC features for 3 genres

In [2]:
def generate_genre_mfccs(genres=3, samples_per_genre=50, max_len=130, n_mfcc=20):
    X, y = [], []
    for genre in range(genres):
        for _ in range(samples_per_genre):
            freq = random.uniform(100, 800) if genre == 0 else random.uniform(400, 1200)
            signal = np.sin(np.linspace(0, 2 * np.pi * freq, 22050))  # Simulated genre tones
            mfcc = librosa.feature.mfcc(y=signal.astype(np.float32), sr=22050, n_mfcc=n_mfcc)
            mfcc = mfcc.T[:max_len]
            if mfcc.shape[0] < max_len:
                pad = max_len - mfcc.shape[0]
                mfcc = np.pad(mfcc, ((0, pad), (0, 0)), mode='constant')
            X.append(mfcc)
            y.append(genre)
    return np.array(X), tf.keras.utils.to_categorical(y, num_classes=genres)

# Generate dataset

In [3]:
X, y = generate_genre_mfccs()
X = X[..., np.newaxis]  # Add channel dimension for CNN

# Train/test split

In [4]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build CNN model

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(y.shape[1], activation='softmax')
])

c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Compile and train

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.4062 - loss: 1.0983 - val_accuracy: 0.0000e+00 - val_loss: 3.7433
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4193 - loss: 0.7372 - val_accuracy: 0.1667 - val_loss: 5.0042
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.5690 - loss: 0.6583 - val_accuracy: 0.1250 - val_loss: 5.7407
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.7174 - loss: 0.5761 - val_accuracy: 0.1250 - val_loss: 6.0999
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.7096 - loss: 0.5337 - val_accuracy: 0.1250 - val_loss: 6.3874
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.7253 - loss: 0.4867 - val_accuracy: 0.1250 - val_loss: 6.6244
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7031 - loss: 0.4755 - val_accuracy: 0.1250 - val_loss: 6.8886
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.7799 - loss: 0.4394 - val_accuracy: 0.1250 - val_loss: 7.

# Evaluate model

In [7]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"🎵 Music Genre Classification Accuracy: {acc:.2f}")

🎵 Music Genre Classification Accuracy: 0.00
